In [1]:
!pip install pyspark==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=5f23db56338853e24324c60b75d76de8f2a8156849993294faac96ca5bee4582
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!pip install findspark

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
import findspark
findspark.init()

In [5]:
# Conectando ao google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
spark = SparkSession.builder.appName("normalizer").getOrCreate()

In [7]:
from pyspark.ml.feature import Normalizer

In [8]:
carros = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Spark/datas/Carros.csv", header=True, inferSchema=True, sep=";")
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [9]:
from pyspark.ml.feature import VectorAssembler
vecassembler = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas"], outputCol="Matriz")
carros_vetor = vecassembler.transform(carros)
carros_vetor.select("Consumo", "Cilindros", "Cilindradas", "Matriz").show()

+-------+---------+-----------+------------------+
|Consumo|Cilindros|Cilindradas|            Matriz|
+-------+---------+-----------+------------------+
|     21|        6|        160|  [21.0,6.0,160.0]|
|     21|        6|        160|  [21.0,6.0,160.0]|
|    228|        4|        108| [228.0,4.0,108.0]|
|    214|        6|        258| [214.0,6.0,258.0]|
|    187|        8|        360| [187.0,8.0,360.0]|
|    181|        6|        225| [181.0,6.0,225.0]|
|    143|        8|        360| [143.0,8.0,360.0]|
|    244|        4|       1467|[244.0,4.0,1467.0]|
|    228|        4|       1408|[228.0,4.0,1408.0]|
|    192|        6|       1676|[192.0,6.0,1676.0]|
|    178|        6|       1676|[178.0,6.0,1676.0]|
|    164|        8|       2758|[164.0,8.0,2758.0]|
|    173|        8|       2758|[173.0,8.0,2758.0]|
|    152|        8|       2758|[152.0,8.0,2758.0]|
|    104|        8|        472| [104.0,8.0,472.0]|
|    104|        8|        460| [104.0,8.0,460.0]|
|    147|        8|        440|

In [10]:
normalizador = Normalizer(inputCol="Matriz", outputCol="Matriznorm", p=1.0)
carrosnorm = normalizador.transform(carros_vetor)
carrosnorm.select("Matriz", "Matriznorm").show(truncate=False)

+------------------+--------------------------------------------------------------+
|Matriz            |Matriznorm                                                    |
+------------------+--------------------------------------------------------------+
|[21.0,6.0,160.0]  |[0.11229946524064172,0.03208556149732621,0.8556149732620321]  |
|[21.0,6.0,160.0]  |[0.11229946524064172,0.03208556149732621,0.8556149732620321]  |
|[228.0,4.0,108.0] |[0.6705882352941176,0.011764705882352941,0.3176470588235294]  |
|[214.0,6.0,258.0] |[0.4476987447698745,0.012552301255230125,0.5397489539748954]  |
|[187.0,8.0,360.0] |[0.33693693693693694,0.014414414414414415,0.6486486486486487] |
|[181.0,6.0,225.0] |[0.4393203883495146,0.014563106796116505,0.5461165048543689]  |
|[143.0,8.0,360.0] |[0.27984344422700586,0.015655577299412915,0.7045009784735812] |
|[244.0,4.0,1467.0]|[0.1422740524781341,0.0023323615160349854,0.8553935860058309] |
|[228.0,4.0,1408.0]|[0.13902439024390245,0.0024390243902439024,0.85853658536